# Web Scraping using BeautifulSoup

### Importing all required packages

In [1]:
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup
import geocoder # import geocoder
#import numpy as np
from geopy.geocoders import Nominatim 
import folium
import os

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url).text

### Collecting required data and converting to dictionary then to dataframe

In [3]:
soup = BeautifulSoup(result, 'lxml')
col = []
values = []
table= soup.find('table', class_ = 'wikitable')

for match in table.find_all('th'):
    head = match.text.rstrip('\n')
    col.append((head,[]))

for item in table.find_all('td'):
    data = item.text.rstrip('\n')
    values.append(data)

for i in range(len(values)):
    col[i%3][1].append(values[i])
    i+=1

Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.tail()

,Postal code,Borough,Neighborhood
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...
179,M9Z,Not assigned,


### Refining the dataframe

In [4]:
df = df[df.Borough!='Not assigned'] # Deleting rows having Not assigned values in Borough column

In [5]:
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [6]:
#df.reset_index(drop_index = True, Inplace = True) # Resetting the index

In [7]:
df['Neighborhood'] = df['Neighborhood'].str.replace('/',',') # Replacing # with ,

In [8]:
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [9]:
df['Neighborhood'] = df['Neighborhood'].str.replace(' , ',', ')

In [10]:
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
df[df['Neighborhood']==''] #Checking if column Neighborhood has any blank values

,Postal code,Borough,Neighborhood


In [12]:
df.shape

(103, 3)

### Uploading geospatial data and merging with original dataframe

In [13]:
geo_df = pd.read_csv('Geospatial_Coordinates.csv') #Creating geo_df with geospatial data
geo_df.rename(columns={"Postal Code":"Postal code"}, inplace = True) # Updating column names so that it will be easy for merge
geo_df.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df = pd.merge(df, geo_df, on ='Postal code', how ='left') # Merging two datafrmaes 

In [15]:
df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Checking unique values of boroughs and neighborhoods

In [16]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


### Finding out coordinates of Toronto city

In [17]:
address = 'Toronto'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


### Creating  map of Toronto and neighborhood points

In [18]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Define Foursquare Credentials and Version

In [22]:
CLIENT_ID = os.environ['CLIENT_ID'] # your Foursquare ID
CLIENT_SECRET = os.environ['CLIENT_SECRET'] # your Foursquare Secret
VERSION = '20200411' # Foursquare API version

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [23]:
toronto_df = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [24]:
toronto_df.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

In [25]:
neighborhood_latitude = toronto_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [26]:
radius = 500
LIMIT = 50

In [27]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
#url

In [28]:
results = requests.get(url).json()['response']['venues']
#a = []
#a.append([b['name'] for b in results])
#a
#results

In [29]:
venues_list = []
for items in results:
    try:
        v_name = items['name']
    except:
        v_name = ''
    try:
        v_lat = items['location']['lat']
    except:
        v_lat = ''
    try:
        v_lng = items['location']['lng'],
    except:
        v_lng = ''
    try:
        v_code = items['location']['postalCode'][:4]
    except:
        v_code = ''
    try:
        v_c_name = items['categories'][0]['name']
    except:
        v_c_name = ''
    venues_list.append([(v_name, v_lat, v_lng, v_code, v_c_name)])
                     
#venues_list

In [30]:
nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])

In [31]:
nearby_venues.columns= ['Venue Name', 'Venue Latitude', 'Venue Longitude','Postal Code', 'Venue Category']

In [32]:
nearby_venues.head()

,Venue Name,Venue Latitude,Venue Longitude,Postal Code,Venue Category
0,City Hall Council Chambers,43.651827,"(-79.38394893163043,)",,City Hall
1,Toronto City Hall,43.653140,"(-79.38396692276001,)",M5H,City Hall
2,GoodLife Fitness Toronto Bell Trinity Centre,43.653436,"(-79.382314,)",M5G,Gym
3,City of Toronto Civic Innovation Office,43.653454,"(-79.383952,)",M5H,City Hall
4,City Hall Podium Green Roof,43.653504,"(-79.38386645704847,)",,Garden
